In [ ]:
import os
import sys
import argparse

print(os.getcwd())
os.chdir("../../")
os.getcwd()

sys.argv = ["view", "--config", "config/multi_task_cross_stitch.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

In [ ]:
from config_experiments import config
from torchvision.transforms import transforms
from dataloader import VOC08Attr
import matplotlib.pyplot as plt
from model import (
    ObjectDetectionModel,
    AttributePredictionModel,
    CrossStitchNet,
    AttributePredictionHead,
    ObjectDetectionHead,
)
from utils import set_device
import torch
from bbox_transform import resize_bounding_boxes, apply_nms
import matplotlib.patches as patches
import torchvision
import torch.nn as nn

In [ ]:
device = "cpu"

model_obj = ObjectDetectionModel().to(device)
model_attr = AttributePredictionModel().to(device)
model = CrossStitchNet(model_obj.backbone, model_attr.backbone).to(device)

In [ ]:
# Lista dei parametri per i due ottimizzatori
params = []
cross_stitch_params = []

# Assegna i parametri ai rispettivi gruppi
for name, param in model.named_parameters():
    if "alfa_a" in name or "alfa_b" in name:
        cross_stitch_params.append(param)
    elif "weight" in name:
        params.append(
            {
                "params": param,
                "lr": config["optimizer"]["lr_global"]
                * config["optimizer"]["lr_weigth_mult"],
                "weight_decay": config["optimizer"]["weight_decay"],
            }
        )
    elif "bias" in name:
        params.append(
            {
                "params": param,
                "lr": config["optimizer"]["lr_global"]
                * config["optimizer"]["lr_bias_mult"],
                "weight_decay": config["optimizer"]["weight_decay"],
            }
        )

# Ottimizzatore per i parametri standard del modello
optimizer = torch.optim.SGD(
    params,
    lr=config["optimizer"]["lr_global"],
    momentum=config["optimizer"]["momentum"],
)

# Ottimizzatore per i parametri cross-stitch
cross_stitch_optimizer = torch.optim.SGD(
    [{"params": cross_stitch_params, "lr": config["cross_stitch"]["lr_cross_stitch"]}],
    momentum=0.0,  # senza momentum
)

In [ ]:
optimizer

In [ ]:
cross_stitch_optimizer

In [ ]:
cross_stitch_params